# Micro Segmentation Postpaid

In [1]:
from __future__ import division, print_function
import pyspark
import pandas as pd
import random
import numpy as np
from pathlib2 import Path
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, SparkConf
conf = pyspark.SparkConf()\
.set("spark.yarn.queue","root.services.Lynx")\
.set("spark.executors.cores","4")\
.set("spark.executor.instances","4")\
.set("spark.executor.memory", "40gb")\
.set("spark.dynamicAllocation.enabled","false")\
.set("spark.driver.memory","20g")\
.set("spark.sql.broadcastTimeout",3600)
spark = SparkSession.builder.config(conf=conf).master("yarn").enableHiveSupport().getOrCreate()
sc = SparkContext.getOrCreate()
from functools import reduce
from collections import OrderedDict
from pyspark.sql.functions import udf,concat, col, lit,substring,max,min,regexp_replace,length,when
from pyspark.sql.types import StringType,IntegerType,TimestampType,DateType 
import dateutil.relativedelta
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

ModuleNotFoundError: No module named 'pyspark'

In [1]:
import calendar
from datetime import datetime, timedelta
import os
import pathlib
import sys
from dateutil.relativedelta import relativedelta
import time

## Micro Segmentation Started 

In [4]:
start_time = time.time()

In [2]:
date_list = ['2019-02-01']
complete_replacement_dict = {}
for input_month in date_list:
    month_input = datetime.strptime(input_month, '%Y-%m-%d')
    previous_month = month_input.replace(day=1) - relativedelta(months=1)
    current_month_input = datetime.strptime(input_month, '%Y-%m-%d').strftime('%Y-%m-%d')

    # current_month_start = datetime.strptime('2017-11-01', '%Y-%m-%d')
    # current_month_end = datetime.strptime('2017-11-30', '%Y-%m-%d')

    current_month_start = datetime.strptime(current_month_input, '%Y-%m-%d')
    print('Current month start:', current_month_start)
    num_of_days_in_month = calendar.monthrange(current_month_start.year, current_month_start.month)[1]
    current_month_end = current_month_start.replace(day=num_of_days_in_month)
    print('Current month end:', current_month_end)

    end_plus_one_day = current_month_end + timedelta(days=1)  # 2017-12-01

    one_month_earlier_start = (current_month_start + timedelta(days=-1)).replace(day=1)  # 2017-10-01
    two_month_earlier_start = (one_month_earlier_start + timedelta(days=-1)).replace(day=1)
    three_month_earlier_start = (two_month_earlier_start + timedelta(days=-1)).replace(day=1)
    four_month_earlier_start = (three_month_earlier_start + timedelta(days=-1)).replace(day=1)

    # var reference
    # $1$ : 20170901
    # $15$ : 20170701
    # $8$ : 20171001
    # $7$ : 20171101
    # $2$ : 20171130
    # $6$ : 20171201
    # $3$ : 2017-09-01
    # $5$ : 2017-11-01
    # $4$ : 2017-12-01

    # $9$ : 201711
    # $10$ : 201710
    # $11$ : 201709
    # $12$ : 201708

    # $13$: 11
    # $14$: 2017

    replacement_dict = {
        '$15$': four_month_earlier_start.strftime('%Y%m%d'),
        '$1$': two_month_earlier_start.strftime('%Y%m%d'),
        '$1_3$': two_month_earlier_start.strftime('%Y-%m-%d'),
        '$2$': current_month_end.strftime('%Y%m%d'),
        '$2_3$': current_month_end.strftime('%Y-%m-%d'),
        '$3$': two_month_earlier_start.strftime('%Y-%m-%d'),
        '$4$': end_plus_one_day.strftime('%Y-%m-%d'),
        '$5$': current_month_start.strftime('%Y-%m-%d'),
        '$6$': end_plus_one_day.strftime('%Y%m%d'),
        '$6_3$': end_plus_one_day.strftime('%Y-%m-%d'),
        '$7$': current_month_start.strftime('%Y%m%d'),
        '$8$': one_month_earlier_start.strftime('%Y%m%d'),
        '$9$': current_month_start.strftime('%Y%m'),
        '$10$': one_month_earlier_start.strftime('%Y%m'),
        '$11$': two_month_earlier_start.strftime('%Y%m'),
        '$12$': three_month_earlier_start.strftime('%Y%m'),
        '$13$': current_month_start.strftime('%m'),
        '$14$': current_month_start.strftime('%Y')}
    complete_replacement_dict[input_month] = replacement_dict

Current month start: 2019-02-01 00:00:00
Current month end: 2019-02-28 00:00:00


In [3]:
replacement_dict

{'$15$': '20181001',
 '$1$': '20181201',
 '$1_3$': '2018-12-01',
 '$2$': '20190228',
 '$2_3$': '2019-02-28',
 '$3$': '2018-12-01',
 '$4$': '2019-03-01',
 '$5$': '2019-02-01',
 '$6$': '20190301',
 '$6_3$': '2019-03-01',
 '$7$': '20190201',
 '$8$': '20190101',
 '$9$': '201902',
 '$10$': '201901',
 '$11$': '201812',
 '$12$': '201811',
 '$13$': '02',
 '$14$': '2019'}

In [7]:
#Create date based on bill_cycle
def create_startend(bill_cycle,service_type,date_input):
    if bill_cycle == 'UNKNOWN' or service_type =='PREPAID':
        bill_cycle = '01'
    bill_date = date_input + '-' + bill_cycle
    bill_date = datetime.strptime(bill_date, '%Y-%m-%d')
    return bill_date

# MicroSegmentation Code Start

In [4]:
post_micro_script = """
WITH POSTPAID_DATA_USAGE_TEMP AS ( 
    SELECT *, ROW_NUMBER() OVER (PARTITION BY billing_subscriber_id,start_date,end_date,product_id
                                    ORDER BY DATA_DATE DESC) RN
    FROM clm.LYNX_PRICING_PCRF_WEEKLY 
    WHERE service_type = 'Postpaid' AND ((DATA_MONTH = '{$9$}') OR (DATA_MONTH = '{$10$}') OR (DATA_MONTH = '{$11$}') OR (DATA_MONTH = '{$12$}'))),

POSTPAID_DATA_USAGE AS (
    SELECT MSISDN,subscriber_arrangement_id, SUM(quota_unit)/1024 AS Quota, SUM(quota_unit)/1024 - SUM(balance_unit)/1024 AS CONSUMPTION, DATA_MONTH
    FROM POSTPAID_DATA_USAGE_TEMP 
    WHERE RN = 1
    GROUP BY MSISDN,subscriber_arrangement_id,DATA_MONTH)
    
SELECT *,
        CASE WHEN (DATA_USED/1024) > 400 THEN 1 ELSE 0 END AS MI_USER_F,
        DATA_RATIO2  AS FRAC_DATA_USED_3_MTH,
        CASE WHEN MAIN_SEGMENT = 'Corporate Corporate' THEN 'Corporate Corporate'
                WHEN MAIN_SEGMENT = 'Staff/Dealer' OR MAIN_SEGMENT = 'Exclude' THEN 'Exclude'
                WHEN MAIN_SEGMENT = 'New' THEN 'New'
                WHEN MAIN_SEGMENT = 'COBP' THEN 'COBP'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 0 
                                             AND IFNULL(NUM_OUT_CALL2, 0) < 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 THEN 'Low Usage, No Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 0 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'High Call, Low Spend, No Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 0 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND SPENDING_RATIO > 1.0625 THEN 'High Call, High Spend, No Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 0 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND (SPENDING_RATIO <= 1.0625) THEN 'High Call, Normal Spend, No Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 1 AND IFNULL(NUM_OUT_CALL2, 0) < 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 THEN 'Low Usage, Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 1 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'High Call, Low Spend, Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 1 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND SPENDING_RATIO > 1.0625 THEN 'High Call, High Spend, Contract'
                WHEN MAIN_SEGMENT = 'Others' AND CONTRACT_IND = 1 AND IFNULL(NUM_OUT_CALL2, 0) >= 20 AND IFNULL(DATA_USED2, 0) / (1024*1024)  < 1.2 AND (SPENDING_RATIO <= 1.0625) THEN 'High Call, Normal Spend, Contract'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON > 0) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'Data, No Contract, Over Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON > 0) AND SPENDING_RATIO > 1.0625  THEN 'Data, No Contract, Over Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON > 0) AND (SPENDING_RATIO <= 1.0625)  THEN 'Data, No Contract, Over Util, Normal Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'Data, No Contract, High Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND SPENDING_RATIO > 1.0625  THEN 'Data, No Contract, High Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND (SPENDING_RATIO <= 1.0625) THEN 'Data, No Contract, High Util, Normal Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'Data, No Contract, Low Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND SPENDING_RATIO > 1.0625  THEN 'Data, No Contract, Low Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 0 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND (SPENDING_RATIO <= 1.0625)  THEN 'Data, No Contract, Low Util, Normal Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON > 0) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0) THEN 'Data, Contract, Over Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON > 0) AND SPENDING_RATIO > 1.0625 THEN 'Data, Contract, Over Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON > 0) AND (SPENDING_RATIO <= 1.0625) THEN 'Data, Contract, Over Util, Normal Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0)  THEN 'Data, Contract, High Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND SPENDING_RATIO > 1.0625  THEN 'Data, Contract, High Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 >= 0.7) AND (SPENDING_RATIO <= 1.0625)  THEN 'Data, Contract, High Util, Normal Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND (SPENDING_RATIO < 0.95 OR BILLED_AMT2=0) THEN 'Data, Contract, Low Util, Low Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND SPENDING_RATIO > 1.0625 THEN 'Data, Contract, Low Util, High Spend'
                WHEN MAIN_SEGMENT = 'Others' AND (IFNULL(DATA_USED2, 0) / (1024*1024))  > 1.2 AND CONTRACT_IND = 1 AND (COUNT_ADD_ON = 0 AND DATA_RATIO2 < 0.7) AND (SPENDING_RATIO <= 1.0625) THEN 'Data, Contract, Low Util, Normal Spend'
                END AS DETAILED_SEGMENT
    FROM
        (SELECT *,
               COUNT_ADD_ON_2GB2 + COUNT_ADD_ON_5GB2 + COUNT_ADD_ON_10GB2 AS COUNT_ADD_ON,
               CASE WHEN BILL_PLAN_NAME2 <> 'LEGACY' THEN 1 ELSE 0 END AS RTP_PLAN,
               CASE WHEN CUST_SEGMENT = '10-CONSUMER' THEN 1 ELSE 0 END AS CUSTOMER_IND,
               CASE WHEN (DEVICE_TYPE = 'BASIC PHONE') or (DEVICE_TYPE = 'FEATURE PHONE') THEN 'NON-SMARTPHONE'
                       WHEN (DEVICE_TYPE = 'TABLETS') THEN 'TABLETS'
                    ELSE 'SMARTPHONE' END AS DEVICE_TYPE_GROUP,
               CASE WHEN (DEVICE_TYPE = 'SMART PHONE' OR DEVICE_TYPE = 'INVALID TAC') THEN 1
                       ELSE 0 END AS SMART_PHONE_IND,
               CASE WHEN AGE < 25 THEN 'YOUTH'
                       WHEN AGE >= 25 AND AGE < 45 THEN 'MATURE I'
                    WHEN AGE >= 45 AND AGE <= 55 THEN 'MATURE II'
                    WHEN AGE >= 55 THEN 'MATURE III' END AS AGE_GROUP,
               CASE WHEN NOT (BRN_NUMBER <> '-1' OR BRN_NUMBER <> 'UNKNOWN') THEN 'Corporate Corporate' --Large cor plan
                    WHEN (BILL_PLAN_NAME LIKE '%TEST%' OR BILL_PLAN_NAME LIKE '%DEMO%'
                          OR BILL_PLAN_NAME LIKE '%DATA%ONLY%' OR BILL_PLAN_NAME LIKE '%M2M%'
                          OR BILL_PLAN_NAME LIKE '%STAFF%' OR BILL_PLAN_NAME LIKE '%VENDOR%'
                          OR BILL_PLAN_NAME LIKE '%CMP%' OR BILL_PLAN_NAME LIKE '%AMBASSADOR%') THEN 'Staff/Dealer'
                    WHEN PRODUCT_GROUP = 'ENTERPRISE_STAFF PACKAGE' OR PRODUCT_GROUP = 'STAFF_DEALER' THEN 'Staff/Dealer'
                    WHEN PRODUCT_GROUP = 'LEGACY' THEN 'Staff/Dealer' --Dealer legacy
                    -- WHEN PRODUCT_GROUP = 'ENTERPRISE_CELCOM FIRST BOX' THEN 'Corporate Exclude'
                    -- WHEN PRODUCT_GROUP = 'UNKNOWN' THEN 'Exclude'  --Large cor plan
                    WHEN BILL_PLAN_NAME2 = 'FIRST 1 + 5' THEN 'Exclude'
                    WHEN BILL_PLAN_NAME = 'SECOND SIM' OR BILL_PLAN_NAME = 'COMPLIMENTARY SIM' OR
                           BILL_PLAN_NAME LIKE '%GBSHARE%' THEN 'Exclude' -- second sim / com sim / gbshare all listed as 0 rev
                    --WHEN BILLED_AMT = 0 AND IMPUTED_AMT_BILLED IS NULL THEN 'Exclude' --Exclude 0 billed amt from any mi targeting
                    WHEN CAST((unix_timestamp(to_date('{$2_3$}'))-unix_timestamp(to_date(ACTIVATION_DATE)))/(30*24*3600)  AS INT) <= 3 THEN 'New'
                    WHEN substring(cast(IFNULL(PLAN_MIGRATION_DATE, ACTIVATION_DATE) AS VARCHAR(20)),6,2) = '{$13$}' AND substring(cast(IFNULL(PLAN_MIGRATION_DATE, ACTIVATION_DATE) AS VARCHAR(20)),1,4) = '{$14$}' THEN 'COBP'
                         ELSE 'Others' END AS MAIN_SEGMENT
        FROM 
            (SELECT t.SUBSCRIBER_ARRANGEMENT_ID, t.MOBILE_NO, PRODUCT_GROUP, '{$9$}' AS DATA_MONTH,COM_SPENDING,
                   PLAN_MIGRATION_DATE AS PLAN_MIGRATION_DATE2,STATUS_TYPE, STATUS_REASON, BILL_PLAN_NAME, PREV_PLAN,
                   ACC_NUMBER, ACC_PARENT, FLAG_IS_PARENT, NUM_SUPPLINE,AGE, BILLED_AMT, DISCOUNT,BRN_NUMBER,
                   (CASE WHEN IMPUTED_AMT_BILLED IS NULL THEN 0 ELSE IMPUTED_AMT_BILLED END) / (CASE WHEN COM_SPENDING IS NULL THEN 2
                                           WHEN COM_SPENDING < 2 THEN 2
                                           ELSE COM_SPENDING END) AS SPENDING_RATIO, --possible /0 err
                   ACTIVATION_DATE,PLAN_MIGRATION_DATE,
                   ACTIVATION_DATE AS ACTIVATION_DATE2,DEVICE_TYPE,CUST_SEGMENT,
                   CAST((unix_timestamp(to_date ('{$2_3$}')) - unix_timestamp(to_date(IFNULL(PLAN_MIGRATION_DATE, ACTIVATION_DATE))))/(30*24*3600)  AS INT) AS SUBSCRIPTION_TENURE,
                   CAST((unix_timestamp(to_date ('{$2_3$}')) - unix_timestamp(to_date(ACTIVATION_DATE)))/(30*24*3600)  AS INT) AS ACTIVATION_TENURE,
                   SUBSTR(BILL_CYCLE, 2, 2) as BILL_DAY,
                   CASE WHEN BILL_PLAN_NAME LIKE '%FIRST%GOLD%SUPREME%' THEN 'FIRST GOLD SUPREME'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%GOLD%PLUS%' THEN 'FIRST GOLD PLUS'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%GOLD%' THEN 'FIRST GOLD'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%BLUE%' THEN 'FIRST BLUE'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%BASIC%' THEN 'FIRST BASIC 38'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%PLATINUM%PLUS%' THEN 'FIRST PLATINUM PLUS'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%PLATINUM%' THEN 'FIRST PLATINUM'
                        WHEN BILL_PLAN_NAME LIKE '%FIRST%1%5' THEN 'FIRST 1 + 5'
                        WHEN BILL_PLAN_NAME LIKE '%XPAX%' THEN 'XPAX'
                        ELSE 'LEGACY' END AS BILL_PLAN_NAME2,
                   CASE WHEN PREV_PLAN LIKE '%FIRST%GOLD%SUPREME%' THEN 'FIRST GOLD SUPREME'
                        WHEN PREV_PLAN LIKE '%FIRST%GOLD%PLUS%' THEN 'FIRST GOLD PLUS'
                        WHEN PREV_PLAN LIKE '%FIRST%GOLD%' THEN 'FIRST GOLD'
                        WHEN PREV_PLAN LIKE '%FIRST%BLUE%' THEN 'FIRST BLUE'
                        WHEN PREV_PLAN LIKE '%FIRST%BASIC%' THEN 'FIRST BASIC 38'
                        WHEN PREV_PLAN LIKE '%FIRST%PLATINUM%PLUS%' THEN 'FIRST PLATINUM PLUS'
                        WHEN PREV_PLAN LIKE '%FIRST%PLATINUM%' THEN 'FIRST PLATINUM'
                        WHEN PREV_PLAN LIKE '%FIRST%1%5' THEN 'FIRST 1 + 5'
                        WHEN PREV_PLAN LIKE '%XPAX%' THEN 'XPAX'
                        WHEN PREV_PLAN LIKE 'UNKNOWN' THEN 'UNKNOWN'
                        ELSE 'LEGACY' END AS PREV_PLAN2,
                   CASE WHEN IMPUTED_AMT_BILLED IS NULL THEN 0 ELSE IMPUTED_AMT_BILLED END AS BILLED_AMT2,
                   IFNULL(CAMPAIGN_TARGET, 0) AS CAMPAIGN_TARGET,
                   DATA_VOLUME, NUM_OUT_SMS2, NUM_OUT_CALL2, NUM_OUT_CALL_IDD, NUM_OUT_CALL_IDD2, 
                   AMT_OUT_CALL_IDD2, ROAMING_FREQ_ACTIVE, DATA_VOLUME2,
                   DATA_USED, DATA_USED2, DATA_UNUSED, DATA_UNUSED2, DATA_RATIO, DATA_RATIO2,
                   IFNULL(COUNT_ADD_ON_2GB,0) AS COUNT_ADD_ON_2GB2,
                   IFNULL(COUNT_ADD_ON_5GB,0) COUNT_ADD_ON_5GB END AS COUNT_ADD_ON_5GB2,
                   IFNULL(COUNT_ADD_ON_10GB,0) AS COUNT_ADD_ON_10GB2,
                   IFNULL(COUNT_ROAMING_PASS,0) AS COUNT_ROAMING_PASS2,
                   contract_end_date AS CONTRACT_DURATION2,
                   contract_type AS CONTRACT_TYPE2,
                   CONTRACT_END_DATE AS CONTRACT_END_DATE2,
                   contract.DEVICE_MODEL AS CONTRACT_DEVICE_MODEL2,
                   CASE WHEN (CONTRACT_FLAG IS NOT NULL) THEN 1
                        ELSE 0 END AS CONTRACT_IND,
                   CAST((unix_timestamp(to_date(CONTRACT_END_DATE))-unix_timestamp(to_date ('{$2_3$}')))/(24*3600)  AS INT) AS DAYS_TO_END_DATE

            FROM (SELECT * FROM cav.cav_postpaid_monthly_tbl WHERE MONTH_KEY = '{$9$}')t
            --Just to replace a imputed bill amt
                LEFT JOIN (SELECT mobile_no no_id,
                            avg(CASE WHEN BILLED_AMT=0 THEN NULL ELSE BILLED_AMT END) AS IMPUTED_AMT_BILLED
                           FROM cav.cav_postpaid_monthly_tbl
                           WHERE (MONTH_KEY='{$10$}' OR MONTH_KEY='{$11$}' OR MONTH_KEY='{$12$}')
                           GROUP BY mobile_no) tb_bill
                ON t.mobile_no = tb_bill.no_id
            -- 3 month usage info
                LEFT JOIN (
                           SELECT MSISDN,
                                  avg(NUM_OUT_CALL_DOMESTIC) AS NUM_OUT_CALL2,
                                  avg(NUM_OUT_SMS_DOMESTIC) AS NUM_OUT_SMS2,
                                  avg(NUM_OUT_CALL_IDD) AS NUM_OUT_CALL_IDD2,
                                  avg(AMT_OUT_CALL_IDD) AS AMT_OUT_CALL_IDD2,
                                  avg(DATA_DOMESTIC_GB) AS DATA_VOLUME2
                           FROM  clm.POSTPAID_USAGE_BY_BILL_CYCLE
                           WHERE (DATA_MONTH='{$10$}' OR DATA_MONTH='{$11$}' OR DATA_MONTH='{$12$}')
                           GROUP BY MSISDN
                ) t2
                ON t.mobile_no = t2.MSISDN
                -- PCRF usage tbl (3m avg)
                    LEFT JOIN (
                               SELECT   MSISDN,
                                        subscriber_arrangement_id,
                                        avg(CONSUMPTION)/1024 AS DATA_USED2,
                                        avg(QUOTA - CONSUMPTION)/1024 AS DATA_UNUSED2,
                                        avg(CONSUMPTION / QUOTA) AS DATA_RATIO2
                               FROM     POSTPAID_DATA_USAGE
                               WHERE    DATA_MONTH='{$10$}' OR DATA_MONTH='{$11$}' OR DATA_MONTH='{$12$}'
                               GROUP BY MSISDN,subscriber_arrangement_id
                    ) t3
                    ON t.subscriber_arrangement_id = t3.subscriber_arrangement_id
                -- PCRF usage tbl (1m lag)
                    LEFT JOIN (
                               SELECT   MSISDN,
                                        subscriber_arrangement_id,
                                        CONSUMPTION/1024 AS DATA_USED,
                                        (QUOTA - CONSUMPTION)/1024 AS DATA_UNUSED,
                                        CONSUMPTION / QUOTA AS DATA_RATIO
                               FROM     POSTPAID_DATA_USAGE
                               WHERE    DATA_MONTH='{$10$}'
                    ) t4
                    ON t.subscriber_arrangement_id = t4.subscriber_arrangement_id
                LEFT JOIN (
                           SELECT SUBSCRIBER_ARR_ID,
                                  sum(COUNT_ADD_ON_5GB)/3 AS COUNT_ADD_ON_5GB,
                                  sum(COUNT_ADD_ON_2GB)/3 AS COUNT_ADD_ON_2GB,
                                  sum(COUNT_ADD_ON_10GB)/3 AS COUNT_ADD_ON_10GB,
                                  sum(COUNT_ROAMING_PASS)/3 AS COUNT_ROAMING_PASS
                           FROM(SELECT 
                                 subscriber_arrangement_id SUBSCRIBER_ARR_ID,
                                    CASE WHEN PRODUCT_ID IN ('1005002','2045892','1003638','45892') THEN 1 ELSE 0 END AS COUNT_ADD_ON_2GB,
                                    CASE WHEN PRODUCT_ID IN ('1005003','2045893','45893','1003639') THEN 1 ELSE 0 END AS COUNT_ADD_ON_5GB,
                                    CASE WHEN PRODUCT_ID IN ('1005004','1003640','2045894','45894') THEN 1 ELSE 0 END AS COUNT_ADD_ON_10GB,
                                    CASE WHEN PRODUCT_ID IN ('1006152','45846','1000045','1001433') THEN 1
                                    ELSE 0 END AS COUNT_ROAMING_PASS
                                   FROM   clm.postpaid_addon_subscription_daily 
                                 WHERE  (TO_DATE(start_date) BETWEEN to_date('{$1_3$}') AND to_date('{$6_3$}'))
                            ) tbl
                            GROUP BY SUBSCRIBER_ARR_ID
                ) addon
                ON t.SUBSCRIBER_ARRANGEMENT_ID = addon.SUBSCRIBER_ARR_ID
            -- Commitment spending by plan
                LEFT JOIN (
                           SELECT BILL_PLAN_NAME AS BPN, COM_SPENDING
                           FROM   clm.POSTPAID_COMMITMENT_SPENDING_201711
                ) com
                ON t.BILL_PLAN_NAME = com.BPN
            -- Phone contract tbl
                LEFT JOIN (
                    SELECT 
                        tb1.msisdn,case when tb2.msisdn is null then tb1.device_model ELSE null END AS device_model,
                        CASE WHEN tb2.msisdn IS NOT NULL THEN tb2.contract_type ELSE tb1.contract_type END AS contract_type,
                        1 as CONTRACT_FLAG,case when tb2.contract_duration is not null then tb2.contract_duration else tb1.contract_duration end as contract_duration,
                        CASE WHEN tb2.month_to_expiry IS NOT NULL THEN tb2.month_to_expiry ELSE tb1.month_to_expiry END AS month_to_expiry,
                        CASE WHEN tb2.end_dt IS NOT NULL THEN tb2.end_dt ELSE tb1.contract_end_date END AS contract_end_date
                        FROM (SELECT
                                    DISTINCT msisdn,contract_type,contract_end_date,device_model,
                                    CAST((unix_timestamp(contract_end_date)-unix_timestamp(to_date(contract_start_date)))/(30*24*3600)  AS INT) AS contract_duration,
                                    CAST((unix_timestamp(contract_end_date)-unix_timestamp(to_date('{$3$}')))/(30*24*3600)  AS INT) AS month_to_expiry
                                FROM aggr.aggr_subscriber_profile_summary_hist_tbl
                                    WHERE contract_expiry not in ('NOT APPLICABLE','NO CONTRACT')
                                    AND status IN ('ACTIVE','CALL BARRING')
                                    AND service_type = 'POSTPAID'
                                    AND to_date(data_day) = to_date('{$3$}')
                                    AND mvno_operator_flag = 'NO'
                                    AND line_of_business <> 'BUSINESS SOLUTION') tb1
                        LEFT JOIN
                            (SELECT 
                                DISTINCT mobile_no msisdn,'LIFE_STYE' As contract_type,end_dt,
                                CAST((unix_timestamp(end_dt)-unix_timestamp(to_date(effective_dt)))/(30*24*3600)  AS INT) AS contract_duration,
                                CAST((unix_timestamp(end_dt)-unix_timestamp(to_date('{$3$}')))/(30*24*3600)  AS INT) AS month_to_expiry
                                FROM
                                cav.cav_pricing_lifestyle_tbl life 
                                INNER JOIN
                                (SELECT DISTINCT MOBILE_NO,SUBSCRIBER_ARRANGEMENT_ID FROM cav.cav_postpaid_daily_tbl cav_post WHERE to_date(DATA_DAY) = to_date('{$3$}'))cav_post
                                ON life.subscriber_arr_id = cav_post.SUBSCRIBER_ARRANGEMENT_ID) tb2
                    ON tb1.msisdn = tb2.msisdn) contract
                ON t.mobile_no = contract.msisdn
                -- Campaign target
                LEFT JOIN (
                           SELECT distinct substr(MSISDN,2,100) MSISDN, 1 as CAMPAIGN_TARGET
                           FROM(
                                SELECT MSISDN, substring(cast(CONTACT_DATE AS VARCHAR(20)),6,2) as t_month
                                FROM   cav.sor_campaign_target_target_tbl
                                WHERE  substring(cast(CONTACT_DATE AS VARCHAR(20)),6,2) = '{$13$}'
                                       AND SUB_TYPE LIKE '%Monetization%'
                                       AND SUB_TYPE LIKE '%Postpaid%'
                                       AND SUB_TYPE NOT LIKE '%OTT%') c1
                        ) cam
                        ON t.mobile_no = cam.MSISDN
            WHERE MONTH_KEY = '{$9$}'
                  AND (LINE_OF_BUSINESS='MOBILE')
                  AND (SUBSCRIPTION_STATUS='ACTIVE' OR SUBSCRIPTION_STATUS='ACTIVE_ACTIVE')
                  AND length(translate(MOBILE_NO_LAST_2DIGITS,'0123456789',''))=0 
                  AND length(MOBILE_NO_LAST_2DIGITS)>0
                  AND DEVICE_TYPE<>'DONGLE'
                  AND flag_mvno='N'
                  AND STATUS_TYPE!='PORT_OUT'
                  AND STATUS_REASON NOT LIKE '%PORT%OUT%') tb_final) complete_segement
"""

In [32]:
for current_month_input in date_list:
    table_exist = str.lower('POSTPAID_MICROSEGMENTATION') in sqlCtx.tableNames('clm')
    if table_exist:
        #insert into
        post_micro = spark.sql(post_micro_script.format(**complete_replacement_dict[current_month_input]))
        post_micro.createOrReplaceTempView("post_micro") 
        spark.sql("""INSERT INTO clm.POSTPAID_MICROSEGMENTATION SELECT * FROM post_micro""")
    else:
        spark.sql(post_micro_script.format(**complete_replacement_dict[current_month_input]))
        post_micro.createOrReplaceTempView("post_micro") 
        spark.sql("""CREATE TABLE clm.POSTPAID_MICROSEGMENTATION_NEW AS SELECT * FROM post_micro""")

DataFrame[]

In [33]:
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

'07:00:44'

Testing Micro Insertion and Creation

In [19]:
spark.sql("SELECT * FROM clm.POSTPAID_MICROSEGMENTATION").limit(10).toPandas()

,SUBSCRIBER_ARRANGEMENT_ID,MOBILE_NO,PRODUCT_GROUP,DATA_MONTH,COM_SPENDING,PLAN_MIGRATION_DATE2,STATUS_TYPE,STATUS_REASON,BILL_PLAN_NAME,PREV_PLAN,ACC_NUMBER,ACC_PARENT,FLAG_IS_PARENT,NUM_SUPPLINE,AGE,BILLED_AMT,DISCOUNT,BRN_NUMBER,SPENDING_RATIO,ACTIVATION_DATE,PLAN_MIGRATION_DATE,ACTIVATION_DATE2,DEVICE_TYPE,CUST_SEGMENT,SUBSCRIPTION_TENURE,ACTIVATION_TENURE,BILL_DAY,BILL_PLAN_NAME2,PREV_PLAN2,BILLED_AMT2,CAMPAIGN_TARGET,DATA_VOLUME,NUM_OUT_SMS2,NUM_OUT_CALL2,NUM_OUT_CALL_IDD,NUM_OUT_CALL_IDD2,AMT_OUT_CALL_IDD2,ROAMING_FREQ_ACTIVE,DATA_VOLUME2,DATA_USED,DATA_USED2,DATA_UNUSED,DATA_UNUSED2,DATA_RATIO,DATA_RATIO2,COUNT_ADD_ON_2GB2,COUNT_ADD_ON_5GB2,COUNT_ADD_ON_10GB2,COUNT_ROAMING_PASS2,CONTRACT_DURATION2,CONTRACT_TYPE2,CONTRACT_END_DATE2,CONTRACT_DEVICE_MODEL2,CONTRACT_IND,DAYS_TO_END_DATE,COUNT_ADD_ON,RTP_PLAN,CUSTOMER_IND,DEVICE_TYPE_GROUP,SMART_PHONE_IND,AGE_GROUP,MAIN_SEGMENT,MI_USER_F,FRAC_DATA_USED_3_MTH,DETAILED_SEGMENT
0,116300000025156319,0102000094,XPAX POSTPAID,201902,27,2019-02-12 15:53:49,PORT IN,UNKNOWN,XPAX 50 2.0,UNKNOWN,338171333,-1,N,0,32,11.6000,0.0000,-1,0E-20,2019-02-12 15:53:49,2019-02-12 15:53:49,2019-02-12 15:53:49,SMART_PHONE,10-CONSUMER,NaN,0,3,XPAX,UNKNOWN,0E-8,None,4922858.9746,NaN,NaN,0,NaN,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaT,None,NaT,None,0,NaN,0.0,1,1,SMARTPHONE,0,MATURE I,New,0,NaN,New
1,211121641765927302,0102003737,FIRST BLUE,201902,56,2018-04-26 17:59:39,CHANGE SIM,UNKNOWN,FIRST BLUE FOR BUSINESS 2.0,UNKNOWN,194599023,303498141,N,0,-1,52.6000,0.0000,904125H,0.85178571428571428571,2018-04-26 00:00:00,2018-04-26 17:59:39,2018-04-26 00:00:00,SMART_PHONE,40-SMI,10.0,10,8,FIRST BLUE,UNKNOWN,47.70000000,None,5129218.0117,NaN,28.000000,0,NaN,None,0,3.5252017933333,3790744.0,5.536459e+06,6695016.0,5.386208e+06,0.361514,0.509455,0.0,0.0,0.0,0.0,2020-04-26 08:00:00,PACKAGE,2020-04-26 08:00:00,iPhone 7 (A1778),1,423.0,0.0,1,0,SMARTPHONE,0,YOUTH,Others,1,0.509455,"Data, Contract, Low Util, Low Spend"
2,190380000016143954,0102028736,XPAX POSTPAID,201902,27,2018-11-10 15:16:02,CHANGE SIM,UNKNOWN,XPAX 50 2.0,UNKNOWN,308331123,-1,N,0,32,50.0000,0.0000,-1,1.61308641962962962963,2018-11-10 15:16:02,2018-11-10 15:16:02,2018-11-10 15:16:02,SMART_PHONE,10-CONSUMER,NaN,3,8,XPAX,UNKNOWN,43.55333333,None,8422549.8145,6.5,79.000000,0,NaN,None,0,3.9625065435000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaT,None,NaT,None,0,NaN,0.0,1,1,SMARTPHONE,0,MATURE I,New,0,NaN,New
3,184940000014602974,0102044213,FIRST GOLD,201902,85,2018-04-24 11:14:07,CHANGE RATE PLAN,CBS MIGRATION,FIRST GOLD,UNKNOWN,302203658,-1,N,0,26,81.9800,0.0000,-1,0.94839215682352941176,2018-04-24 11:14:07,2018-04-24 11:14:07,2018-04-24 11:14:07,SMART_PHONE,10-CONSUMER,NaN,10,8,FIRST GOLD,UNKNOWN,80.61333333,None,19932284.8945,NaN,174.000000,0,NaN,None,0,22.8951541143333,4542744.0,7.775633e+06,16428776.0,1.406970e+07,0.216615,0.352443,0.0,0.0,0.0,0.0,2037-01-01 00:00:00,LIFE_STYE,2037-01-01 00:00:00,None,1,6517.0,0.0,1,1,SMARTPHONE,0,MATURE I,Others,1,0.352443,"Data, Contract, Low Util, Low Spend"
4,141910000014370753,0102085982,FIRST GOLD,201902,85,2018-03-16 18:05:13,PORT IN,UNKNOWN,FIRST GOLD,UNKNOWN,301442968,-1,N,0,37,80.0000,0.0000,-1,1.07290196082352941176,2018-03-16 18:05:13,2018-03-16 18:05:13,2018-03-16 18:05:13,SMART_PHONE,10-CONSUMER,NaN,11,8,FIRST GOLD,UNKNOWN,91.19666667,None,2221441.6953,1.0,62.333333,0,NaN,None,0,1.7330350900000,996786.0,8.801067e+05,19974734.0,2.096523e+07,0.047530,0.040615,0.0,0.0,0.0,0.0,NaT,None,NaT,None,0,NaN,0.0,1,1,SMARTPHONE,0,MATURE I,Others,1,0.040615,"High Call, High Spend, No Contract"
5,136710000022629060,0102186500,FIRST GOLD PLUS,201902,104,2019-01-18 08:54:26,PORT IN,UNKNOWN,FIRST GOLD PLUS FOR BUSINESS,UNKNOWN,329832851,-1,N,0,28,89.8100,-20.0000,-1,0.31153846153846153846,2019-01-18 08:54:26,2019-01-18 08:54:26,2019-01-18 08:54:26,SMART_PHONE,30-GOVERNMENT,NaN,1,8,FIRST GOLD PLUS,UNKNOWN,32.40000000,None,40425972.1123,NaN,NaN,0,NaN,None,0,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaT,

In [20]:
spark.sql("SELECT * FROM clm.POSTPAID_MICROSEGMENTATION").count()

2276058

In [21]:
spark.stop()